<a href="https://colab.research.google.com/github/anhquanpham/TracIn/blob/master/imagenet/resnet50_imagenet_proponents_opponents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A-Mazing Proponents and Opponents [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/frederick0329/TrackIn/blob/master/imagenet/resnet50_imagenet_proponents_opponents.ipynb)



*   ![A-Maze of interest](https://user-images.githubusercontent.com/3983652/86517275-8892d380-bddc-11ea-83b0-eea69408a4cd.jpeg)
*   This colab shows an application of [TrackIn](https://arxiv.org/abs/2002.08484) - Proponents and Opponents in the training data of an example given a function of interest. The function of interest in this colab is the loss function of the example.
*   This colab allows you to find examples in training data which would decrease (proponents) or increase (opponents) the loss function of a given example.
*   Imagenet subset (10%) is used due to colab default RAM size (12G). To run the full imagenet, around 40G of RAM is required.     

To Run

*   Runtime -> Change Runtime Type -> TPU


To adapt to your own data/model
  *   Load your own model and select checkpoints.
  *   Pick weights.
  *   Replace loss function.
  *   Write your own dataset loader.
  *   [FAQ](https://drive.google.com/file/d/1zL3hwW4wFru49_-zwpmliRDdCahjumXa/view)







# Define varaibles

In [2]:
IMAGENET_TFDS_DIR = "tensorflow_datasets" #@param {type:"string"}
IMAGENET_TRAIN = "imagenet/train" #@param {type:"string"}
IMAGENET_VAL = "imagenet/validation" #@param {type:"string"}
CHECKPOINTS_PATH_FORMAT = "ckpt{}" #@param {type:"string"}

## Auth if your data is stored on gcs

In [3]:
from google.colab import auth
auth.authenticate_user()

# Clone git

In [4]:
!git clone https://github.com/frederick0329/TrackIn

Cloning into 'TrackIn'...
remote: Enumerating objects: 150, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 150 (delta 32), reused 9 (delta 5), pack-reused 90 (from 1)
Receiving objects: 100% (150/150), 69.61 MiB | 21.10 MiB/s, done.
Resolving deltas: 100% (72/72), done.


# Import and Utils

In [5]:
!pip install tensorflow==2.0.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 284.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.3/475.3 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 87.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 8.2 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.17.2
    Uninstalling wrapt-1.17.2:
      Successfully uninstalled wrapt-1.17.2
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uni

In [6]:
# @title Imports
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.image as mpimg
import io
import json
import numpy as np
import time
import datetime
import matplotlib.pyplot as plt
import functools
import sys
sys.path.insert(0, "./TrackIn/imagenet/resnet50")
import resnet

In [7]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
#@title Connect to GPU
try:
  # Try to connect to a GPU, if available
  gpu = tf.config.list_physical_devices('GPU')
  if gpu:
    print('Running on GPU ', gpu[0])
    strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
  else:
    raise ValueError('ERROR: Not connected to a GPU runtime; please see the previous cell in this notebook for instructions!')

except ValueError as e:
  raise BaseException('ERROR: Not connected to a GPU runtime; please see the previous cell in this notebook for instructions!') from e

# tf.config.experimental_connect_to_cluster(tpu) # This line is specific to TPU and should be removed
# tf.tpu.experimental.initialize_tpu_system(tpu) # This line is specific to TPU and should be removed
# strategy = tf.distribute.experimental.TPUStrategy(tpu) # This line is specific to TPU and should be removed

In [9]:
#@title Dataset Utils

def _train_filename2id(filename):
  filename = tf.strings.regex_replace(filename, "n", "")
  filename = tf.strings.regex_replace(filename, ".JPEG", "")
  filename_split = tf.strings.split(filename, "_")
  fileid = tf.strings.to_number(filename_split, tf.int32)
  return fileid

def _val_filename2id(filename):
  filename = tf.strings.regex_replace(filename, "ILSVRC2012_val_", "")
  filename = tf.strings.regex_replace(filename, ".JPEG", "")
  fileid = tf.strings.to_number(filename, tf.int32)
  return fileid

def _resize_image(image_bytes: tf.Tensor,
                 height: int = 224,
                 width: int = 224) -> tf.Tensor:
  """Resizes an image to a given height and width."""
  return tf.compat.v1.image.resize(
      image_bytes, [height, width], method=tf.image.ResizeMethod.BILINEAR,
      align_corners=False)

# Calculated from the ImageNet training set
MEAN_RGB = (0.485 * 255, 0.456 * 255, 0.406 * 255)
STDDEV_RGB = (0.229 * 255, 0.224 * 255, 0.225 * 255)

def mean_image_subtraction(
    image_bytes,
    means,
    num_channels = 3,
    dtype = tf.float32,
):
  """Subtracts the given means from each image channel.

  For example:
    means = [123.68, 116.779, 103.939]
    image_bytes = mean_image_subtraction(image_bytes, means)

  Note that the rank of `image` must be known.

  Args:
    image_bytes: a tensor of size [height, width, C].
    means: a C-vector of values to subtract from each channel.
    num_channels: number of color channels in the image that will be distorted.
    dtype: the dtype to convert the images to. Set to `None` to skip conversion.

  Returns:
    the centered image.

  Raises:
    ValueError: If the rank of `image` is unknown, if `image` has a rank other
      than three or if the number of channels in `image` doesn't match the
      number of values in `means`.
  """
  if image_bytes.get_shape().ndims != 3:
    raise ValueError('Input must be of size [height, width, C>0]')

  if len(means) != num_channels:
    raise ValueError('len(means) must match the number of channels')

  # We have a 1-D tensor of means; convert to 3-D.
  # Note(b/130245863): we explicitly call `broadcast` instead of simply
  # expanding dimensions for better performance.
  means = tf.broadcast_to(means, tf.shape(image_bytes))
  if dtype is not None:
    means = tf.cast(means, dtype=dtype)

  return image_bytes - means


def standardize_image(
    image_bytes,
    stddev,
    num_channels = 3,
    dtype = tf.float32,
):
  """Divides the given stddev from each image channel.

  For example:
    stddev = [123.68, 116.779, 103.939]
    image_bytes = standardize_image(image_bytes, stddev)

  Note that the rank of `image` must be known.

  Args:
    image_bytes: a tensor of size [height, width, C].
    stddev: a C-vector of values to divide from each channel.
    num_channels: number of color channels in the image that will be distorted.
    dtype: the dtype to convert the images to. Set to `None` to skip conversion.

  Returns:
    the centered image.

  Raises:
    ValueError: If the rank of `image` is unknown, if `image` has a rank other
      than three or if the number of channels in `image` doesn't match the
      number of values in `stddev`.
  """
  if image_bytes.get_shape().ndims != 3:
    raise ValueError('Input must be of size [height, width, C>0]')

  if len(stddev) != num_channels:
    raise ValueError('len(stddev) must match the number of channels')

  # We have a 1-D tensor of stddev; convert to 3-D.
  # Note(b/130245863): we explicitly call `broadcast` instead of simply
  # expanding dimensions for better performance.
  stddev = tf.broadcast_to(stddev, tf.shape(image_bytes))
  if dtype is not None:
    stddev = tf.cast(stddev, dtype=dtype)

  return image_bytes / stddev

# TPU does not allow tf.string and images with various size. Therefore, decode
# and cropping cannot happen in the model.
def _preprocess(inputs, split='train', image_size=224, crop_padding=32):
  """Apply image preprocessing."""
  filename = inputs['file_name']
  image = inputs['image']
  label = inputs['label']
  if split == 'train':
    fileid = _train_filename2id(filename)
  else:
    fileid = _val_filename2id(filename)
  shape = tf.shape(image)
  image_height = shape[0]
  image_width = shape[1]

  padded_center_crop_size = tf.cast(
      ((image_size / (image_size + crop_padding)) *
       tf.cast(tf.minimum(image_height, image_width), tf.float32)),
      tf.int32)

  offset_height = ((image_height - padded_center_crop_size) + 1) // 2
  offset_width = ((image_width - padded_center_crop_size) + 1) // 2
  crop_window = tf.stack([offset_height, offset_width,
                          padded_center_crop_size, padded_center_crop_size])

  image = tf.image.crop_to_bounding_box(
        image,
        offset_height=offset_height,
        offset_width=offset_width,
        target_height=padded_center_crop_size,
        target_width=padded_center_crop_size)
  image = _resize_image(image_bytes=image,
                        height=image_size,
                        width=image_size)
  image = mean_image_subtraction(
        image, MEAN_RGB)
  image = standardize_image(
        image, STDDEV_RGB)

  image = tf.cast(image, tf.float32)
  label = tf.cast(label, tf.int32)
  return fileid, image, label


def make_get_dataset(split, batch_size):
  def get_dataset(
      input_context: tf.distribute.InputContext = None) -> tf.data.Dataset:
    builder = tfds.builder(name='imagenet2012_subset/10pct', data_dir=IMAGENET_TFDS_DIR)
    builder.download_and_prepare()

    read_config = tfds.ReadConfig(
        interleave_block_length=1)

    _preprocess_fn = functools.partial(_preprocess, split=split)

    ds = builder.as_dataset(
        split=split,
        as_supervised=False,
        shuffle_files=False,
        read_config=read_config)
    ds = ds.map(_preprocess_fn,
                num_parallel_calls=tf.data.experimental.AUTOTUNE)
    ds = ds.batch(batch_size)

    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
    return ds
  return get_dataset

In [10]:
#@title Search Utils

def find(loss_grad=None, activation=None, topk=50):
  if loss_grad is None and activation is None:
    raise ValueError('loss grad and activation cannot both be None.')
  scores = []
  scores_lg = []
  scores_a = []
  for i in range(len(trackin_train['image_ids'])):
    if loss_grad is not None and activation is not None:
      lg_sim = np.sum(trackin_train['loss_grads'][i] * loss_grad, axis=0)
      a_sim = np.sum(trackin_train['activations'][i] * activation, axis=0)
      scores.append(np.sum(lg_sim * a_sim))
      scores_lg.append(np.sum(lg_sim))
      scores_a.append(np.sum(a_sim))
    elif loss_grad is not None:
      scores.append(np.sum(trackin_train['loss_grads'][i] * loss_grad))
    elif activation is not None:
      scores.append(np.sum(trackin_train['activations'][i] * activation))

  opponents = []
  proponents = []
  indices = np.argsort(scores)
  for i in range(topk):
    index = indices[-i-1]
    proponents.append((
        trackin_train['image_ids'][index],
        trackin_train['probs'][index][0],
        index_to_classname[str(trackin_train['predicted_labels'][index][0])][1],
        index_to_classname[str(trackin_train['labels'][index])][1],
        scores[index],
        scores_lg[index] if scores_lg else None,
        scores_a[index] if scores_a else None))
    index = indices[i]
    opponents.append((
        trackin_train['image_ids'][index],
        trackin_train['probs'][index][0],
        index_to_classname[str(trackin_train['predicted_labels'][index][0])][1],
        index_to_classname[str(trackin_train['labels'][index])][1],
        scores[index],
        scores_lg[index] if scores_lg else None,
        scores_a[index] if scores_a else None))
  return opponents, proponents

IMAGENET_LABEL_DICT = './TrackIn/imagenet/imagenet_class_index.json'
def get_id_synset_mapping():
  imagenet_class_idx_path = IMAGENET_LABEL_DICT
  with tf.io.gfile.GFile(imagenet_class_idx_path, "r") as f:
    json_str = f.read()
    index_to_classname = json.loads(json_str)
  return index_to_classname
index_to_classname = get_id_synset_mapping()

def get_image(split, id):
  if split == 'validation':
    filepath = '{}/ILSVRC2012_val_{fileid:08d}.JPEG'.format(IMAGENET_VAL, fileid=id)
    print('ILSVRC2012_val_{fileid:08d}.JPEG'.format(fileid=id))
  else:
    filepath = '{}/n0{}/n0{}_{}.JPEG'.format(IMAGENET_TRAIN, id[0], id[0], id[1])
    print('n0{}_{}.JPEG'.format(id[0], id[1]))
  try:
    with tf.io.gfile.GFile(filepath, "rb") as f:
      jpg_data = f.read()
      image = mpimg.imread(io.BytesIO(jpg_data), format='JPG')
    return image
  except:
    print('Failed to read image {}'.format(filepath))

def find_and_show(trackin_dict, idx, vector='influence', idx_filename_mapping=None):
  if vector == 'influence':
    op, pp = find(trackin_dict['loss_grads'][idx], trackin_dict['activations'][idx])
  elif vector == 'encoding':
    op, pp = find(None, trackin_dict['activations'][idx])
  elif vector == 'error':
    op, pp = find(trackin_dict['loss_grads'][idx], None)
  else:
    raise ValueError('Unsupported vector type.')
  print('Query image from validation: ')
  print('label: {}, prob: {}, predicted_label: {}'.format(
      index_to_classname[str(trackin_dict['labels'][idx])][1],
      trackin_dict['probs'][idx][0],
      index_to_classname[str(trackin_dict['predicted_labels'][idx][0])][1]))
  if idx_filename_mapping:
    img = mpimg.imread(io.BytesIO(idx_filename_mapping[idx]), format='JPG')
  else:
    img = get_image('validation', trackin_dict['image_ids'][idx])
  if img is not None:
    plt.imshow(img, interpolation='nearest')
    plt.show()
  print("="*50)
  print('Proponents: ')
  for p in pp:
    print('label: {}, prob: {}, predicted_label: {}, influence: {}'.format(p[3], p[1], p[2], p[4]))
    if p[5] and p[6]:
      print('error_similarity: {}, encoding_similarity: {}'.format(p[5], p[6]))
    img = get_image('train', p[0])
    if img is not None:
      plt.imshow(img, interpolation='nearest')
      plt.show()
  print("="*50)
  print('Opponents: ')
  for o in op:
    print('label: {}, prob: {}, predicted_label: {}, influence: {}'.format(o[3], o[1], o[2], o[4]))
    if o[5] and o[6]:
      print('error_similarity: {}, encoding_similarity: {}'.format(o[5], o[6]))
    img = get_image('train', o[0])
    if img is not None:
      plt.imshow(img, interpolation='nearest')
      plt.show()
  print("="*50)

# Init models/checkpoints



* Resnet50 - 75.8% accuracy
* Pick 30, 60, 90th checkpoint
* Model checkpoint weights are converted to saved models so the colab does not depend on model code.



In [11]:
with strategy.scope():
  models_penultimate = []
  models_last = []
  for i in [30, 60, 90]:
    model = resnet.resnet50(1000)
    model.load_weights(CHECKPOINTS_PATH_FORMAT.format(i))
    models_penultimate.append(tf.keras.Model(model.layers[0].input, model.layers[-3].output))
    models_last.append(model.layers[-2])

ModuleNotFoundError: No module named 'keras._tf_keras'

# Find Proponents and Opponents for a given test example

In [ ]:
@tf.function
def run(inputs):
  imageids, images, labels = inputs
  # ignore bias for simplicity
  loss_grads = []
  activations = []
  for mp, ml in zip(models_penultimate, models_last):
    h = mp(images)
    logits = ml(h)
    probs = tf.nn.softmax(logits)
    loss_grad = tf.one_hot(labels, 1000) - probs
    activations.append(h)
    loss_grads.append(loss_grad)

  # Using probs from last checkpoint
  probs, predicted_labels = tf.math.top_k(probs, k=1)

  return imageids, tf.stack(loss_grads, axis=-1), tf.stack(activations, axis=-1), labels, probs, predicted_labels

In [ ]:
def get_trackin_grad(ds):
  image_ids_np = []
  loss_grads_np = []
  activations_np = []
  labels_np = []
  probs_np = []
  predicted_labels_np = []
  for d in ds:
    imageids_replicas, loss_grads_replica, activations_replica, labels_replica, probs_replica, predictied_labels_replica = strategy.run(run, args=(d,))
    for imageids, loss_grads, activations, labels, probs, predicted_labels in zip(
        strategy.experimental_local_results(imageids_replicas),
        strategy.experimental_local_results(loss_grads_replica),
        strategy.experimental_local_results(activations_replica),
        strategy.experimental_local_results(labels_replica),
        strategy.experimental_local_results(probs_replica),
        strategy.experimental_local_results(predictied_labels_replica)):
      if imageids.shape[0] == 0:
        continue
      image_ids_np.append(imageids.numpy())
      loss_grads_np.append(loss_grads.numpy())
      activations_np.append(activations.numpy())
      labels_np.append(labels.numpy())
      probs_np.append(probs.numpy())
      predicted_labels_np.append(predicted_labels.numpy())
  return {'image_ids': np.concatenate(image_ids_np),
          'loss_grads': np.concatenate(loss_grads_np),
          'activations': np.concatenate(activations_np),
          'labels': np.concatenate(labels_np),
          'probs': np.concatenate(probs_np),
          'predicted_labels': np.concatenate(predicted_labels_np)
         }

## Build Imagenet Train Vectors



In [ ]:
ds_train = strategy.experimental_distribute_datasets_from_function(make_get_dataset('train', 512))

In [ ]:
start = time.time()
trackin_train = get_trackin_grad(ds_train)
end = time.time()
print(datetime.timedelta(seconds=end - start))

## Build Imagenet Val Vectors

In [ ]:
ds_val = strategy.experimental_distribute_datasets_from_function(make_get_dataset('validation', 512))

In [ ]:
start = time.time()
trackin_val = get_trackin_grad(ds_val)
end = time.time()
print(datetime.timedelta(seconds=end - start))

## Pick an index from Validation Set

In [ ]:
find_and_show(trackin_val, 83, 'influence')